<div style="text-align: right">Paul Novaes<br>August 2018</div> 

# Le compte est bon

[Le compte est bon](https://en.wikipedia.org/wiki/Des_chiffres_et_des_lettres#Le_compte_est_bon_&#40;%22the_total_is_right%22&#41;) ("The total is right") is a French TV show where the goal is to get as close as possible to some target number by using 6 randomly drawn numbers and the 4 arithmetic operations.

For example, if the target is __383__ and the numbers are __1, 3, 7, 7, 9, 25__, one possible solution (not necessarily the best) is: 

$$25 \times (7 + 9) - 7 \times (3 - 1) = 386$$

In this notebook, we write a program that finds the best solution, using the least amount of numbers. We then explore some variations of the original game and estimate their difficulty.

## Rules of the Game

* The target is a random number between $101$ and $999$.
* The 6 other numbers are drawn at random, without replacement from: $$\{1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100\}$$
* Each of the 6 numbers can only be used once, and not all numbers need to be used.
* Temporary and final results have to be positive integers.

## Random draw

Python's random.sample function makes drawing a random game very easy:

In [1]:
import random

# For consistency.
random.seed(0)

def draw(n = 6, lo = 101, hi = 999, cards = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100]):
    target = random.randint(lo, hi)
    return target, sorted(random.sample(cards, n))

For example:

In [2]:
target, cards = draw()
print('target: ', target, ', cards: ', cards, sep = '')

target: 965, cards: [1, 5, 7, 7, 8, 9]


## Best solution with exactly n cards

We can solve this problem recursively as follows. 

If there is just 1 card, use that card. Otherwise, if we have $n > 1$ cards:
* choose 2 of the cards $a$ and $b$ and 1 of the operators $op$
* replace $a$ and $b$ by $a$ op $b$
* solve the problem on the resulting $n - 1$ cards

Note that $a+b$ and $a \times b$ are commutative and $a - b$ and $a \div b$ are only allowed if $a \geq b$. Since there are ${n \choose 2}$ ways of choosing 2 cards and 4 ways of choosing 1 operator, the total number of operations $N(n)$ verifies, for $n > 2$:

$$N(n) \leq 4{n \choose 2} N(n-1)$$

The first values are:

\begin{align*}
N(2) & \leq & 4 \\
N(3) & \leq & 48 \\
N(4) & \leq & 1152 \\
N(5) & \leq & 46080 \\
N(6) & \leq & 2764800
\end{align*}

Though large, these numbers are __not too__ large and we can do an extensive search.

In [3]:
# Returns the best solution using all cards.
def solve_using_all(target, cards, ops = ['+', '-', '*', '/']):
    count = len(cards)
    assert count > 0
    if len(cards) == 1:        
        return cards[0], [cards[0]]
    best_reached = 0
    best_solution = []
    # Choose 2 cards and 1 operator.
    for i in range(count):
        for j in range(i + 1, count):
            for k in ops:
                a = cards[i]
                b = cards[j]
                new_cards = list(cards)
                del new_cards[j], new_cards[i]
                if a < b: a, b = b, a
                if k == '+': result = a + b
                if k == '-': result = a - b
                if k == '*': result = a * b
                if k == '/':
                    if b > 0 and a % b == 0:
                        result = a // b
                    else:
                        continue
                new_cards.append(result)
                reached, solution = solve_using_all(target, new_cards, ops)
                if abs(reached - target) < abs(best_reached - target):
                    best_reached = reached
                    best_solution = [[a, k, b, '=', result]]
                    best_solution.extend(solution)
                if reached == target:
                    return best_reached, best_solution
    return best_reached, best_solution

For example:

In [4]:
target, cards = draw()
print(target, cards)
best_reached, best_solution = solve_using_all(target, cards)
print(best_solution)

515 [4, 5, 6, 8, 9, 10]
[[6, '*', 4, '=', 24], [24, '-', 10, '=', 14], [14, '*', 8, '=', 112], [112, '-', 9, '=', 103], [103, '*', 5, '=', 515], 515]


Or we can print the solution in a more natural way:

In [5]:
def print_solution(solution):
    for operation in solution:
        if type(operation) is int:
            if len(solution) == 1:
                print(operation)
        else:
            for token in operation:
                print(token, end = ' ')
        print()

print_solution(best_solution)

6 * 4 = 24 
24 - 10 = 14 
14 * 8 = 112 
112 - 9 = 103 
103 * 5 = 515 



__Some Unit Tests__

In [6]:
def test(target, cards, expect):
    reached, solution = solve_using_all(target, cards)
    assert(abs(target - reached) == abs(target - expect))

def unit_tests():
    test(1, [1], 1)
    test(1, [2], 2)
    test(1, [3], 3)
    test(10, [1, 1], 2)
    test(10, [1, 1, 1], 3)
    test(10, [1, 1, 1, 1], 4)
    test(10, [1, 1, 1, 1, 1], 6)
    test(10, [1, 1, 1, 1, 1, 1], 9)
    test(5, [1, 1, 1, 1, 1, 1], 5)
    test(1, [1, 1, 1, 1, 1, 1], 1)
    test(2, [1, 1, 1, 1, 1, 1], 2)

unit_tests()

## Shortest Solution

Now that we know how to compute the best solution that uses exactly $n$ cards, we can compute, among the best solutions, the shortest one. We use again a brute force approach by looking at subsets with 1, 2, 3, 4, 5 and 6 elements. In total, the number of operations is bounded by

$${6 \choose 2}N(2) + {6 \choose 3}N(3) + {6 \choose 4}N(4) + {6 \choose 5}N(5) + {6 \choose 6}N(6) = 3059580$$ 


In [7]:
# Returns the number of 1's in the binary representation of n.
def count_1_bits(n):
    count = 0
    while n > 0:
        if n & 1 != 0:
            count += 1
        n = n >> 1
    return count

# Returns a subset of a set, by using mask to select its elements.
def make_subset(set, mask):
    subset = []
    index = 0
    while mask > 0:
        if mask & 1 != 0:
            subset.append(set[index])
        index += 1
        mask = mask >> 1
    return subset

# Returns the best solution using as few cards as possible.
def solve_using_any(target, all_cards, ops = ['+', '-', '*', '/']):
    best_reached = 0
    best_solution = []
    n = len(all_cards)
    # Solve the problem with 1 card first, then 2, then 3...
    for num_cards in range(1, n + 1):
        for mask in range(2 ** n):
            if count_1_bits(mask) == num_cards:
                cards = make_subset(all_cards, mask)
                reached, solution = solve_using_all(target, cards, ops)
                if abs(reached - target) < abs(best_reached - target):
                    best_reached = reached
                    best_solution = solution
                if reached == target:
                    return best_reached, best_solution
    return best_reached, best_solution

# Alias.
def solve(target, cards, ops = ['+', '-', '*', '/']):
    return solve_using_any(target, cards, ops)

__Some Unit Tests__

In [8]:
def test_shortest(target, cards, expect, length):
    reached, solution = solve(target, cards)
    assert(abs(target - reached) == abs(target - expect))
    assert(length == len(solution))
    
def unit_tests_shortest():
    test_shortest(1, [1], 1, 1)
    test_shortest(1, [2], 0, 0)
    test_shortest(1, [3], 0, 0)
    test_shortest(10, [1, 1], 2, 2)
    test_shortest(10, [1, 1, 1], 3, 3)
    test_shortest(10, [1, 1, 1, 1], 4, 4)
    test_shortest(10, [1, 1, 1, 1, 1], 6, 5)
    test_shortest(10, [1, 1, 1, 1, 1, 1], 9, 6)
    test_shortest(5, [1, 1, 1, 1, 1, 1], 5, 5)
    test_shortest(1, [1, 1, 1, 1, 1, 1], 1, 1)
    test_shortest(2, [1, 1, 1, 1, 1, 1], 2, 2)

unit_tests_shortest()

## Solving our original game

In [9]:
target, cards = 383, [1, 3, 7, 7, 9, 25]
print('target =', target)
print('cards =', cards)
print()
reached, solution = solve(target, cards)
print_solution(solution)

target = 383
cards = [1, 3, 7, 7, 9, 25]

9 + 3 = 12 
25 + 7 = 32 
32 * 12 = 384 
384 - 1 = 383 



## Most difficult game

[http://patquoi.free.fr/lcpdb/](http://patquoi.free.fr/lcpdb/) defines the most difficult game as the game that:
1. can be solved in 1 way only
2. requires the 6 cards
3. and, among the games that verify 1. and 2., has the highest intermediary result

and concludes, after trying all the possibilities, that it is:
* target: 653
* cards: {10, 10, 25, 50, 75, 100}

Let's solve this game:

In [10]:
target, cards = 653, [10, 10, 25, 50, 75, 100]
reached, solution = solve(target, cards)
print_solution(solution)

50 * 10 = 500 
500 - 10 = 490 
490 * 100 = 49000 
49000 - 25 = 48975 
48975 / 75 = 653 



## Variations

With the rules of the original show, around 93% of the games can be solved exactly. 

We now investigate how changing the rules affects the odds. For each variation, we will play 100 random games and compute how many we can solve exactly. Since the sample is relatively small, the estimate will be rough but it will give us a way of comparing different variations.

In [11]:
def score(count = 100, n = 6, lo = 101, hi = 999, 
          cards = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100], ops = ['+', '-', '*', '/']):
    success = 0
    random.seed(0)
    for i in range(count):
        target, drawn = draw(n, lo, hi, cards)
        reached, solution = solve(target, drawn, ops)
        if reached == target:
            success += 1
    return str(success * 100 / count) + '%'

__Original rules__

With the original rules, around 93 percent of games can be solved. Since we use a small sample of 100 random games, our estimate below is a little off, but close enough:

In [12]:
default_score = score()
print("original rules: ", default_score)

original rules:  90.0%


__Changing the number of cards drawn__

Unsurprisingly, the more cards we draw the easier the game:

In [13]:
print("2 cards:", score(n = 2))
print("3 cards:", score(n = 3))
print("4 cards:", score(n = 4))
print("5 cards:", score(n = 5))
print("6 cards:", default_score)
print("7 cards:", score(n = 7))

2 cards: 0.0%
3 cards: 1.0%
4 cards: 15.0%
5 cards: 45.0%
6 cards: 90.0%
7 cards: 100.0%


__Limiting the set of operations__

$\times$ is essential while $\div$ is seldom necessary:

In [14]:
print("no * op:", score(ops = ['+', '-', '/']))
print("no + op:", score(ops = ['-', '*', '/']))
print("no - op:", score(ops = ['+', '*', '/']))
print("no / op:", score(ops = ['+', '-', '*']))
print("all ops:", default_score)

no * op: 3.0%
no + op: 57.0%
no - op: 72.0%
no / op: 89.0%
all ops: 90.0%


__Varying the target range__

Bigger targets are quite a bit more difficult to reach:

In [15]:
print("target range 701-999: ", score(lo=701, hi=999))
print("target range 401-700: ", score(lo=401, hi=700))
print("target range 101-400: ", score(lo=101, hi=400))

target range 701-999:  81.0%
target range 401-700:  91.0%
target range 101-400:  99.0%


Because, bigger targets are more difficult to reach, it appears the distribution of cards is not ideal in the original game and $\{1,2,3,4,5,6,7,8,9,10,25,50,75,100\}$ would be better:

In [16]:
print("alternate distribution: ", score(cards = [1,2,3,4,5,6,7,8,9,10,25,50,75,100]))

alternate distribution:  99.0%


## "Countdown"

The British version of "Le compte est bon" is [Countdown](https://en.wikipedia.org/wiki/Countdown_%28game_show%29). This version allows players to vary the distribution by choosing the number of "large" cards.

Players choose a number $k$ between 0 and 4. Then, randomly, $k$ "large" cards (from $\{25,50,75,100\}$) are drawn together with $6-k$ "small" cards (from $\{1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10\}$).


In [17]:
def draw_countdown(k):
    assert k >= 0 and k <= 4
    large_set = [25,50,75,100]
    small_set = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10]
    target = random.randint(101, 999)
    aset = random.sample(large_set, k)
    aset.extend(random.sample(small_set, 6 - k))
    return target, sorted(aset)
    

def score_countdown(count = 100, k = 0):
    success = 0
    random.seed(0)
    for i in range(count):
        target, drawn = draw_countdown(k)
        reached, solution = solve(target, drawn)
        if reached == target:
            success += 1
    return str(success * 100 / count) + '%'

for i in range(0,5):
    print(i, "large cards:", score_countdown(k = i))

0 large cards: 77.0%
1 large cards: 97.0%
2 large cards: 100.0%
3 large cards: 97.0%
4 large cards: 87.0%
